# Importowanie niezbędnych bibliotek

In [1]:
import os
import numpy as np
import pandas as pd
from skimage import io, color, img_as_ubyte
from skimage.feature import graycomatrix, graycoprops
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

# Przygotowanie próbek tekstur

In [2]:
def crop_and_save_images(image_dir, output_dir, crop_size=(128, 128)):
    for category in os.listdir(image_dir):
        category_path = os.path.join(image_dir, category)
        if os.path.isdir(category_path):
            for image_file in os.listdir(category_path):
                if image_file.endswith(('.png', '.jpg', '.jpeg')):
                    image_path = os.path.join(category_path, image_file)
                    image = io.imread(image_path)
                    height, width = image.shape[:2]
                    for i in range(0, height, crop_size[0]):
                        for j in range(0, width, crop_size[1]):
                            if i + crop_size[0] <= height and j + crop_size[1] <= width:
                                cropped = image[i:i + crop_size[0], j:j + crop_size[1]]
                                category_dir = os.path.join(output_dir, category)
                                os.makedirs(category_dir, exist_ok=True)
                                fragment_index = len(os.listdir(category_dir))
                                fragment_path = os.path.join(category_dir, f"sample_{category}_{fragment_index}.png")
                                io.imsave(fragment_path, cropped)
                else :
                    print(f"Skipping non-image file: {image_file}")

textures_dir = "Textures"
samples_dir = "Samples"
crop_and_save_images(image_dir=textures_dir, output_dir=samples_dir, crop_size=(128, 128))
print("Cropped images have been saved to the Samples folder.")

c:\Users\mullt\anaconda3\envs\POI\Lib\site-packages\skimage\_shared\utils.py:328: UserWarning: Samples\Gres\sample_Gres_119.png is a low contrast image
  return func(*args, **kwargs)


Cropped images have been saved to the Samples folder.


# Wczytywanie próbek i wyznaczanie cech tekstur

In [ ]:
def extract_and_save_features_from_samples(samples_dir, output_csv):
    feature_vectors = []
    distances = [1, 3, 5]
    angles = [0, np.pi/4, np.pi/2, 3*np.pi/4]
    properties = ['dissimilarity', 'correlation', 'contrast', 'energy', 'homogeneity', 'ASM']
    categories = []

    for category in os.listdir(samples_dir):
        category_path = os.path.join(samples_dir, category)
        if os.path.isdir(category_path):
            for image_file in os.listdir(category_path):
                if image_file.endswith(('.png', '.jpg', '.jpeg')):
                    image_path = os.path.join(category_path, image_file)
                    image = io.imread(image_path)

                    if image.shape[-1] == 4:
                        image = color.rgba2rgb(image)

                    if len(image.shape) == 3:
                        image = color.rgb2gray(image)
                    
                    image = img_as_ubyte(image)
                    glcm = graycomatrix(image, distances=distances, angles=angles, symmetric=True, normed=True)
                    features = []
                    for prop in properties:
                        for d in range(len(distances)):
                            for a in range(len(angles)):
                                features.append(graycoprops(glcm, prop)[d, a])
                    features.append(category)
                    feature_vectors.append(features)
                    categories.append(category)

    # Define column names for the CSV
    columns = [f"{prop}_d{d}_a{a}" for prop in properties
               for d in range(len(distances)) for a in range(len(angles))]
    columns.append("category")

    # Save features to CSV
    df = pd.DataFrame(feature_vectors, columns=columns)
    df.to_csv(output_csv, index=False)
    print(f"Features have been saved to {output_csv}")

output_csv = "texture_features.csv"
extract_and_save_features_from_samples(samples_dir, output_csv)

# Trenowanie i testowanie

In [26]:
def train_and_test_classifier(feature_csv, test_size = 0.2, random_state = 42):
    data = pd.read_csv(feature_csv)
    X = data.drop('category', axis=1).values

    y = data['category'].values
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state)
    classifier = SVC(kernel='linear')
    classifier.fit(X_train, y_train)
    y_pred = classifier.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    print(f"Classifier Accuracy: {accuracy * 100:.2f}%")

train_and_test_classifier(output_csv)

Classifier Accuracy: 100.00%
